In [5]:
import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset
import time
import glob

import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm_notebook

In [30]:
train_path = glob.glob('./train_set/*/*.png')
np.random.shuffle(train_path)

train_label_index = [
    "Speed limit (5km)",         # 0
    "Speed limit (15km)",        # 1
    "Speed limit (20km)",        # 2
    "Speed limit (30km)",        # 3
    "Speed limit (40km)",        # 4
    "Speed limit (50km)",        # 5
    "Speed limit (60km)",        # 6
    "Speed limit (70km)",        # 7
    "Speed limit (80km)",        # 8
    "Speed limit (100km)",       # 9
    "Speed limit (120km)",       # 10
    "End of speed limit",        # 11
    "End of speed limit (50km)", # 12
    "End of speed limit (80km)", # 13
    "Dont overtake from Left",   # 14
    "No stopping",               # 15
    "No Uturn",                  # 16
    "No Car",                    # 17
    "No horn",                   # 18
    "No entry",                  # 19
    "No passage",                # 20
    "Dont Go Right",             # 21
    "Dont Go Left or Right",     # 22
    "Dont Go Left",              # 23
    "Dont Go straight",          # 24
    "Dont Go straight or Right", # 25
    "Dont Go straight or left",  # 26
    "Go right or straight",      # 27
    "Go left or straight",       # 28
    "Village",                   # 29
    "Uturn",                     # 30
    "ZigZag Curve",              # 31
    "Bicycles crossing",         # 32
    "Keep Right",                # 33
    "Keep Left",                 # 34
    "Roundabout mandatory",      # 35
    "Watch out for cars",        # 36
    "Slow down and give way",    # 37
    "Continuous detours",        # 38
    "Slow walking",              # 39
    "Horn",                      # 40
    "Uphill steep slope",        # 41
    "Downhill steep slope",      # 42
    "Under Construction",        # 43
    "Heavy Vehicle Accidents",   # 44
    "Parking inspection",        # 45
    "Stop at intersection",      # 46
    "Train Crossing",            # 47
    "Fences",                    # 48
    "Dangerous curve to the right", # 49
    "Go Right",                  # 50
    "Go Left or right",          # 51
    "Dangerous curve to the left", # 52
    "Go Left",                   # 53
    "Go straight",               # 54
    "Go straight or right",      # 55
    "Children crossing",         # 56
    "Care bicycles crossing",    # 57
    "Danger Ahead",              # 58
    "Traffic signals",           # 59
    "Zebra Crossing",            # 60
    "Road Divider"               # 61
]
train_label_index = [x.lower() for x in train_label_index]
train_label = [train_label_index.index(x.split('/')[-2].lower()) for x in train_path]

test_path = glob.glob('./test_set/*.png')
test_path.sort()

In [31]:
len(train_path)

9808

In [32]:
test_path[:10]

['./test_set/005d3487-92a5-48f3-9784-379f1217825d.png',
 './test_set/00a6a262-1643-443f-b8ed-7fc075ac3c1e.png',
 './test_set/00f7f66d-96f6-4c9c-b476-dced05e7211f.png',
 './test_set/01a4038f-159e-4a73-bc65-2d20b3cc2a2d.png',
 './test_set/01cf52dd-4235-403b-9696-00490042dcbd.png',
 './test_set/01d298fe-8d03-4ad4-9547-362c9c982438.png',
 './test_set/01ed9a77-80a1-40ba-a14f-434d8cae1fc4.png',
 './test_set/01f3eadb-adc3-4676-afa6-41558ffcee0f.png',
 './test_set/025d6acb-021f-4dd2-9fc2-6a6ea1c25d7f.png',
 './test_set/027848aa-c988-4fe8-a9d7-cedbe2671e2e.png']

In [33]:
len(train_label_index)

62

In [34]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

class ProgressMeter(object):
    def __init__(self, num_batches, *meters):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = ""


    def pr2int(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'
def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    progress = ProgressMeter(len(val_loader), batch_time, losses, top1)

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (input, target) in tqdm_notebook(enumerate(val_loader), total=len(val_loader)):
            input = input.cuda()
            target = target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc = (output.argmax(1).view(-1) == target.float().view(-1)).float().mean() * 100
            losses.update(loss.item(), input.size(0))
            top1.update(acc, input.size(0))
            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f}'
              .format(top1=top1))
        return top1

def predict(test_loader, model, tta=10):
    # switch to evaluate mode
    model.eval()
    
    test_pred_tta = None
    for _ in range(tta):
        test_pred = []
        with torch.no_grad():
            end = time.time()
            for i, (input, target) in tqdm_notebook(enumerate(test_loader), total=len(test_loader)):
                input = input.cuda()
                target = target.cuda()

                # compute output
                output = model(input)
                output = F.softmax(output, dim=1)
                output = output.data.cpu().numpy()

                test_pred.append(output)
        test_pred = np.vstack(test_pred)
    
        if test_pred_tta is None:
            test_pred_tta = test_pred
        else:
            test_pred_tta += test_pred
    
    return test_pred_tta

def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    progress = ProgressMeter(len(train_loader), batch_time, losses, top1)

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        input = input.cuda(non_blocking=True)
        target = target.cuda(non_blocking=True)

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        losses.update(loss.item(), input.size(0))

        acc = (output.argmax(1).view(-1) == target.float().view(-1)).float().mean() * 100
        top1.update(acc, input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 100 == 0:
            progress.pr2int(i)

In [35]:
class XFDataset(Dataset):
    def __init__(self, img_path, img_label, transform=None):
        self.img_path = img_path
        self.img_label = img_label
        
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None
    
    def __getitem__(self, index):
        img = Image.open(self.img_path[index]).convert('RGB')
        
        if self.transform is not None:
            img = self.transform(img)
        
        return img, torch.from_numpy(np.array(self.img_label[index]))
    
    def __len__(self):
        return len(self.img_path)

In [38]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

import timm
model = timm.create_model('resnet18', pretrained=True, num_classes=62)
model = model.cuda()

In [39]:
train_loader = torch.utils.data.DataLoader(
    XFDataset(train_path[:-500], train_label[:-500], 
            transforms.Compose([
                        transforms.Resize((256, 256)),
                        transforms.RandomHorizontalFlip(),
                        transforms.RandomVerticalFlip(),
                        transforms.ColorJitter(brightness=.5, hue=.3),
                        transforms.ToTensor(),
                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    ), batch_size=20, shuffle=True, num_workers=4, pin_memory=True
)

val_loader = torch.utils.data.DataLoader(
    XFDataset(train_path[-500:], train_label[-500:], 
            transforms.Compose([
                        transforms.Resize((256, 256)),
                        transforms.ToTensor(),
                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    ), batch_size=20, shuffle=False, num_workers=4, pin_memory=True
)

criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.Adam(model.parameters(), 0.005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.85)
best_acc = 0.0
for epoch in range(10):
    scheduler.step()
    print('Epoch: ', epoch)

    train(train_loader, model, criterion, optimizer, epoch)
    val_acc = validate(val_loader, model, criterion)
    
    if val_acc.avg.item() > best_acc:
        best_acc = round(val_acc.avg.item(), 2)

Epoch:  0


/home/lyz/anaconda3/envs/py311/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


[  0/466]	Time  0.299 ( 0.299)	Loss 4.0628e+00 (4.0628e+00)	Acc@1   0.00 (  0.00)
[100/466]	Time  0.048 ( 0.046)	Loss 1.5523e+00 (2.6152e+00)	Acc@1  40.00 ( 29.60)
[200/466]	Time  0.043 ( 0.044)	Loss 7.8758e-01 (1.9598e+00)	Acc@1  75.00 ( 45.05)
[300/466]	Time  0.044 ( 0.044)	Loss 4.9413e-01 (1.5744e+00)	Acc@1  85.00 ( 55.40)
[400/466]	Time  0.039 ( 0.044)	Loss 5.3654e-01 (1.3211e+00)	Acc@1  75.00 ( 61.97)


/tmp/ipykernel_2510832/2676678360.py:51: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, (input, target) in tqdm_notebook(enumerate(val_loader), total=len(val_loader)):


  0%|          | 0/25 [00:00<?, ?it/s]

...

 * Acc@1 98.200
Epoch:  8
[  0/466]	Time  0.300 ( 0.300)	Loss 5.4997e-02 (5.4997e-02)	Acc@1  95.00 ( 95.00)
[100/466]	Time  0.043 ( 0.046)	Loss 2.2293e-02 (4.9052e-02)	Acc@1 100.00 ( 98.42)
[200/466]	Time  0.042 ( 0.045)	Loss 5.4809e-02 (5.5094e-02)	Acc@1  95.00 ( 98.06)
[300/466]	Time  0.044 ( 0.044)	Loss 3.8192e-03 (5.2097e-02)	Acc@1 100.00 ( 98.16)
[400/466]	Time  0.044 ( 0.044)	Loss 2.8451e-04 (4.7325e-02)	Acc@1 100.00 ( 98.32)


  0%|          | 0/25 [00:00<?, ?it/s]

 * Acc@1 98.400
Epoch:  9
[  0/466]	Time  0.299 ( 0.299)	Loss 1.4325e-03 (1.4325e-03)	Acc@1 100.00 (100.00)
[100/466]	Time  0.044 ( 0.046)	Loss 1.4930e-02 (2.6899e-02)	Acc@1 100.00 ( 99.11)
[200/466]	Time  0.043 ( 0.045)	Loss 1.2194e-02 (4.1103e-02)	Acc@1 100.00 ( 98.73)
[300/466]	Time  0.044 ( 0.044)	Loss 2.5995e-02 (5.4468e-02)	Acc@1 100.00 ( 98.41)
[400/466]	Time  0.048 ( 0.044)	Loss 8.7019e-03 (5.1813e-02)	Acc@1 100.00 ( 98.45)


  0%|          | 0/25 [00:00<?, ?it/s]

 * Acc@1 98.600


In [42]:
test_loader = torch.utils.data.DataLoader(
    XFDataset(test_path, [0] * len(test_path), 
            transforms.Compose([
                        transforms.Resize((256, 256)),
                        transforms.ToTensor(),
                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    ), batch_size=40, shuffle=False, num_workers=4, pin_memory=True
)

val_label = pd.DataFrame()
val_label['ImageID'] = [x.split('/')[-1] for x in test_path]
val_label['label'] = predict(test_loader, model, 1).argmax(1)
val_label.to_csv('submit.csv', index=None)

/tmp/ipykernel_2510832/2676678360.py:81: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, (input, target) in tqdm_notebook(enumerate(test_loader), total=len(test_loader)):


  0%|          | 0/25 [00:00<?, ?it/s]

In [43]:
val_label

,ImageID,label
0,005d3487-92a5-48f3-9784-379f1217825d.png,2
1,00a6a262-1643-443f-b8ed-7fc075ac3c1e.png,20
2,00f7f66d-96f6-4c9c-b476-dced05e7211f.png,6
3,01a4038f-159e-4a73-bc65-2d20b3cc2a2d.png,17
4,01cf52dd-4235-403b-9696-00490042dcbd.png,4
...,...,...
963,fe028821-9968-473c-8d47-836ff3e6699a.png,14
964,fe41a271-2bba-40aa-b153-14a54be4d8a1.png,21
965,feaee038-cbed-4fbe-a7ab-1ab491807a7c.png,13
966,fef7660d-2c03-40da-b0c3-7bfc916d4a56.png,18
